In [1]:
import asyncio

from azure.identity.aio import DefaultAzureCredential
from azure.ai.projects.models import BingGroundingTool


from semantic_kernel.agents.azure_ai import AzureAIAgent, AzureAIAgentSettings, agent_content_generation
from semantic_kernel.contents import AuthorRole, ChatMessageContent

ImportError: cannot import name 'BingGroundingTool' from 'azure.ai.projects.models' (c:\Users\mutaza\AppData\Local\miniconda3\envs\ai_env\lib\site-packages\azure\ai\projects\models\__init__.py)

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

True

In [3]:
# Global variable to store citation URLs
citation_url = "No citations available"

# Save the original function
original_generate_annotation_content = agent_content_generation.generate_annotation_content
def patched_generate_annotation_content(annotation):
    global citation_url
    print("Patched function called")  # Debug print

    # Ensure that the annotation has both 'start_index' and 'end_index' attributes.
    # print(f"# Annotation before: '{annotation}'")
    if not hasattr(annotation, "start_index"):
        setattr(annotation, "start_index", None)
        # print(f"# Annotation set start_index: '{annotation}'")
    if not hasattr(annotation, "end_index"):
        setattr(annotation, "end_index", None)
        # print(f"# Annotation set end_index: '{annotation}'")

    # print(f"# Annotation after: '{annotation}'")
    # Check if this is a URL citation annotation and expose the URL
    if hasattr(annotation, 'type') and annotation.type == 'url_citation':
        try:
            citation_url = annotation['url_citation']['url']
            # print(f"# Citation url: '{citation_url}'")
            return annotation
        except Exception as e:
            print(f"Error processing citation: {e}")
    # Return the annotation as is for other annotation types
    return annotation

# Apply the monkey-patch
agent_content_generation.generate_annotation_content = patched_generate_annotation_content

print("Bing citation patch function run")  # Debug print

Bing citation patch function run


In [4]:
# Simulate a conversation with the agent
USER_INPUTS = [
    "What is the weather in seattle?",
]

In [5]:
chatmessage = ChatMessageContent(
    role=AuthorRole.USER,
    content=USER_INPUTS[0],
)


chatmessage

ChatMessageContent(inner_content=None, ai_model_id=None, metadata={}, content_type='message', role=<AuthorRole.USER: 'user'>, name=None, items=[TextContent(inner_content=None, ai_model_id=None, metadata={}, content_type='text', text='What is the weather in seattle?', encoding=None)], encoding=None, finish_reason=None)

In [6]:
ai_agent_settings = AzureAIAgentSettings.create()

In [7]:
ai_agent_settings

AzureAIAgentSettings(env_file_path=None, env_file_encoding='utf-8', model_deployment_name='gpt-4.1', project_connection_string=SecretStr('**********'), endpoint=None, subscription_id=None, resource_group_name=None, project_name=None)

In [8]:
ai_agent_settings.project_connection_string

SecretStr('**********')

In [9]:
client = AzureAIAgent.create_client(
    credential=DefaultAzureCredential(),
    conn_str=os.getenv("AZURE_AI_AGENT_PROJECT_CONNECTION_STRING"),
)

In [10]:

creds = DefaultAzureCredential()


In [11]:

# Get your Bing connection
bing_connection = await client.connections.get(
        connection_name=os.getenv("AZURE_AI_AGENT_BING_CONNECTION_NAME"),
        include_credentials=True  # Optional. Defaults to "False"
)
conn_id = bing_connection.id
print(conn_id)


/subscriptions/e9b9c71e-d2ba-4086-a5ed-2923e15962d9/resourceGroups/rg-mutaza-5102_ai/providers/Microsoft.MachineLearningServices/workspaces/ai-inference-eastus2/connections/magbinggrounding01


In [12]:

# Initialize the Bing Search tool
bing = BingGroundingTool(connection_id=conn_id)


In [ ]:

# 1. Create an agent using Azure AI agent service
agent_definition = await client.agents.create_agent(
    model=os.getenv("AZURE_AI_AGENT_MODEL_NAME", "gpt-4.1"),
    name="BingAgent",
    instructions="Answer the user's questions.",
    tools=bing.definitions,
    headers={"x-ms-enable-preview": "true"},
)
print("Created agent definition")


Created agent definition


In [16]:
from semantic_kernel import __version__ as sk_version
print(f"Semantic Kernel version: {sk_version}")

Semantic Kernel version: 1.21.3


In [ ]:
from semantic_kernel.agents.azure_ai.azure_ai_agent import azure
USER_INPUTS = ["Hello", "What's the capital of france?"]

thread: AzureAIAgentThread = AzureAIAgentThread()

try:
    for user_input in USER_INPUTS:
        response = await agent.get_response(messages=user_inputs, thread=thread)
        print(response)
        thread = response.thread
finally:
    await thread.delete() if thread else None

In [14]:

# 2. Create a Semantic Kernel agent for the Azure AI agent
agent = AzureAIAgent(
    client=client,
    definition=agent_definition,
)


In [ ]:
agent.add_chat_

In [ ]:

# 3. Create a new thread on the Azure AI agent service
thread = await client.agents.create_thread()

await agent.add_chat_message(
    thread_id=thread.id,
    message=ChatMessageContent(
        role=AuthorRole.USER,
        content=USER_INPUTS[0],
    )
)
print(f"# User: '{USER_INPUTS[0]}'")



# User: 'What is the weather in seattle?'


In [54]:
response = agent.invoke(
    thread_id=thread.id,

)

In [ ]:
results = []
async for r in  response:

    results.append(r)


ValueError: Invalid combination of arguments provided.

In [38]:
response = await agent.get_response(thread_id=thread.id)
print(f"# {response.name}: {response}")

AttributeError: 'AzureAIAgent' object has no attribute 'get_response'

In [ ]:

try:
    for user_input in USER_INPUTS:

        # 4. Add the user input as a chat message
        await agent.add_chat_message(thread_id=thread.id, message=user_input)
        print(f"# User: '{user_input}'")


        # 5. Invoke the agent for the specified thread for response
        response = await agent.get_response(thread_id=thread.id)
        print(f"# {response.name}: {response}")

            # Retrieve run step details to get Bing Search query link
            # This code is not working so it is commented out
        # runs = await client.agents.list_runs(thread_id=thread.id)
        # print(f"# The most recent run: '{runs}'")

        # run_id = runs['last_id']
        # print(f"# The most recent run_id: '{run_id}'")


        # run_steps = await client.agents.list_run_steps(run_id = run_id, thread_id=thread.id)
        # for run_step in run_steps['data']:
        #     if 'bing_search_query' in run_step:
        #         search_query = run_step['bing_search_query']
        #         bing_search_url = f"https://www.bing.com/search?q={search_query.replace(' ', '+')}"
        #         print(f"Bing Search URL: {bing_search_url}")
        # Print the captured citation URL
        print(f"Citation URL: {citation_url}")


finally:
    # 6. Cleanup: Delete the thread and agent
    await client.agents.delete_thread(thread.id)
    # Do not clean up the assistant so it can be used again

In [11]:
await main()

ValueError: HTTP transport has already been closed. You may check if you're calling a function outside of the `async with` of your client creation, or if you called `await close()` on your client already.